In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [ ]:
# Folder
mission_path = os.path.join(Path().home(), "digiforest_mission_data/latest")

In [ ]:
# Other params
base_inverted = True

%matplotlib inline

## Matplotlib config

In [ ]:
cm = 1 / 2.54
plot_width = 8.89 * cm
plot_height = 4 * cm

plt.rcParams["font.size"] = 8

## Load data

### Load pose files

In [ ]:
from scipy.spatial.transform import Rotation as R


def read_poses_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep='first')]

    # Parse data
    poses = {}
    for ts, x, y, z, qx, qy, qz, qw in zip(
        df.index, df["x"], df["y"], df["z"], df["qx"], df["qy"], df["qz"], df["qw"]
    ):
        poses[f"{ts:.10f}"] = np.eye(4)
        poses[f"{ts:.10f}"][0:3, 3] = np.array([x, y, z])
        poses[f"{ts:.10f}"][0:3, 0:3] = R.from_quat([qx, qy, qz, qw]).as_matrix()

        # TODO: fix base inversion

    return df, poses


df_state_poses, poses_list = read_poses_file(
    os.path.join(mission_path, "states/state_pose_data.csv"), base_inverted=base_inverted
)

### Load twist files

In [ ]:
def read_twist_file(filename, base_inverted=False):
    df = pd.read_csv(filename)
    df = df.drop_duplicates()

    # Generate timestamp from sec and nsec
    ts = 1e9 * df["sec"] + df["nsec"]  # In nanoseconds
    df.index = pd.to_datetime(ts)
    df = df[~df.index.duplicated(keep='first')]

    # Correct twist due to base inversion
    if base_inverted:
        df["vx"] *= -1
        df["vy"] *= -1

    # Speeds
    df["lin_speed"] = (df["vx"] ** 2 + df["vy"] ** 2).pow(1.0 / 2)
    df["ang_speed"] = df["wz"].abs()

    return df


df_state_twist = read_twist_file(
    os.path.join(mission_path, "states/state_twist_data.csv"), base_inverted=base_inverted
)
df_reference_twist = read_twist_file(
    os.path.join(mission_path, "states/reference_twist_data.csv"), base_inverted=base_inverted
)
df_operator_twist = read_twist_file(
    os.path.join(mission_path, "states/operator_twist_data.csv"), base_inverted=base_inverted
)

### Read other operator signals

In [ ]:
reference_frames = ["base_vilens", "map_vilens"]
query_frames = ["LF_FOOT", "RF_FOOT", "LH_FOOT", "RH_FOOT"]

df_tf = {}
for parent in reference_frames:
    for child in query_frames:
        prefix = f"{parent}_{child}"
        df, poses = read_poses_file(
            os.path.join(mission_path, f"states/{prefix}_data.csv"), base_inverted=base_inverted
        )
        
        df_tf[prefix] = df.copy(deep=True).drop_duplicates()

In [ ]:
# Join all indices
joined_indices = pd.Index(df_tf[prefix].index)
for k,v in df_tf.items():
  joined_indices = joined_indices.union(v.index)

joined_indices = joined_indices.drop_duplicates()

for k,v in df_tf.items():
  df_tf[k] = df_tf[k].reindex(index=joined_indices, )
  df_tf[k] = df_tf[k].interpolate(method="index")

In [ ]:
# Plotting
plot_width = 20 * cm
plot_height = 5 * cm
fig, ax = plt.subplots(
    1, 1, figsize=(plot_width, plot_height), constrained_layout=False, dpi=300
)
ax.set_aspect('equal')

import scipy.signal as signal

# plt.plot(df_tf["base_vilens_LF_FOOT"]["z"][200:500].rolling(1, center=True).median())
# plt.plot(df_tf["base_vilens_LF_FOOT"]["z"][200:500].diff().rolling(3, center=True).mean())
# plt.plot(df_tf["base_vilens_LF_FOOT"]["z"][0:300].rolling(3, center=True).mean())
# plt.plot(df_tf["base_vilens_RF_FOOT"]["z"][100:300])
# plt.plot(df_tf["base_vilens_LH_FOOT"]["z"][100:300])
# plt.plot(df_tf["base_vilens_RH_FOOT"]["z"][100:300])

total_footsteps = 0
footsteps = {}
for foot in ["LF_FOOT", "RF_FOOT", "LH_FOOT", "RH_FOOT"]:
    prefix = f"base_vilens_{foot}"
    off = 0
    dt = -1
    y = df_tf[prefix]["z"][off:off+dt]
    sy = df_tf[prefix]["z"].rolling(5, center=True).mean()[off:off+dt]
    t = df_tf[prefix]["z"].index[off:off+dt]

    peaks, _ = signal.find_peaks(-sy, distance=10, prominence=0.01)
    footsteps[foot] = peaks
    # ax.plot(t, y, linewidth=0.5)
    # ax.plot(t, sy, linewidth=0.5)
    # ax.plot(t[peaks], y[peaks], marker="x", linewidth=0)
    total_footsteps += peaks.size

    print(f"num_footsteps: {peaks.size}")

for foot,v in foot_steps.items():
    prefix = f"map_vilens_{foot}"
    ax.scatter(df_tf[prefix]["x"][v], df_tf[prefix]["y"][v], s=2, marker="o", edgecolor="none", alpha=0.5)

foot_radius = 0.03 # meters
foot_area = np.pi * foot_radius**2

total_footprint = total_footsteps * foot_area
print(f"steps: {total_footsteps}")
print(f"area_impacted: {total_footprint:.4f} m^2")

# nout = 100
# w = np.linspace(0.001, 10, nout)
# pgram = signal.lombscargle(df_tf["base_vilens_LF_FOOT"]["z"].index[0:300], df_tf["base_vilens_LF_FOOT"]["z"][0:300], w)
# plt.plot(pgram)
# plt.show()
# pgram


### Load TF files

### Load SLAM graph
Required for distance computation and coverage

In [ ]:
df_slam, slam_graph = read_poses_file(
    os.path.join(mission_path, "states/slam_graph_data.csv")
)

### Align time series

In [ ]:
# Extend indices
joined_indices = df_state_twist.index.union(df_reference_twist.index).drop_duplicates()
joined_indices = joined_indices.union(df_operator_twist.index).drop_duplicates()

df_state_twist = df_state_twist.reindex(index=joined_indices)
df_state_twist = df_state_twist.interpolate(method="index")

df_reference_twist = df_reference_twist.reindex(index=joined_indices)
df_reference_twist = df_reference_twist.interpolate(method="index")

## Estimate metrics

### Operator interventions
This is required for autonomy metrics

In [ ]:
# Interventions from safety officer
df_state_twist["safety_intervention"] = df_operator_twist["lin_speed"] > 0.01
df_state_twist["safety_intervention"] = df_state_twist[
    "safety_intervention"
].interpolate(method="pad")

# Interventions from forestry operator


# Total interventions
df_state_twist["interventions"] = df_state_twist["safety_intervention"]

#### Estimate covered area

In [ ]:
odom_points = df_state_poses[["x", "y"]].to_numpy()
slam_points = df_slam[["x", "y"]].to_numpy()

import shapely
import shapely.plotting

sp_odom_points = shapely.MultiPoint(odom_points)

sp_slam_points = shapely.MultiPoint(slam_points)
sp_slam_hull = sp_slam_points.convex_hull

sensor_range = 30 / 2

sp_sensing_hull = sp_slam_hull.buffer(sensor_range)

#### 

### Normalize time

In [ ]:
ref_ts = df_state_twist.index[0]

df_state_poses.index = df_state_poses.index - ref_ts
df_state_twist.index = df_state_twist.index - ref_ts
df_reference_twist.index = df_reference_twist.index - ref_ts
df_operator_twist.index = df_operator_twist.index - ref_ts
df_slam.index = df_slam.index - ref_ts

### Mission statistics

In [ ]:
import scipy.integrate as integrate

stats = {}

# Constants
stats["sqm_to_ha"] = 0.0001
stats["sec_to_min"] = 1 / 60
stats["sec_to_hour"] = 1 / 3600

# Speed
stats["max_lin_speed"] = df_state_twist["lin_speed"].max().item()
stats["min_lin_speed"] = df_state_twist["lin_speed"].min().item()
stats["mean_lin_speed"] = df_state_twist["lin_speed"].mean().item()
stats["std_lin_speed"] = df_state_twist["lin_speed"].std().item()

stats["mean_ang_speed"] = df_state_twist["ang_speed"].mean().item()
stats["std_ang_speed"] = df_state_twist["ang_speed"].std().item()

# Distance walked
stats["distance_m"] = (
    df_slam[["x", "y", "z"]]
    .diff()
    .apply(lambda values: sum([v**2 for v in values]), axis=1)
    .sum()
).item()

# Mission time
stats["time_sec"] = (df_slam.index[-1] - df_slam.index[0]).total_seconds()

# Interventions
stats["interventions"] = integrate.simpson(
    df_state_twist["interventions"], df_state_twist.index.total_seconds()
).item()

# Percentaje of interventions
stats["interventions_perc"] = stats["interventions"] / stats["time_sec"] * 100

# Area covered
stats["area_m2"] = sp_slam_hull.area
stats["area_ha"] = sp_slam_hull.area * stats["sqm_to_ha"]
stats["sensor_range_m"] = sensor_range
stats["sensed_area_m2"] = sp_sensing_hull.area
stats["sensed_area_ha"] = sp_sensing_hull.area * stats["sqm_to_ha"]

# Hectares per second
stats["ha_per_sec"] = stats["area_m2"] / stats["time_sec"]
stats["ha_per_min"] = stats["area_m2"] / (stats["time_sec"] * stats["sec_to_min"])
stats["ha_per_hour"] = stats["area_m2"] / (stats["time_sec"] * stats["sec_to_hour"])

# Print
for k, v in stats.items():
    print(f"{k:>20}: {v:<20.4f}")


# Save as YAML
import yaml
file=open(os.path.join(mission_path, "mission_report.yaml"),"w")
yaml.dump(stats, file)
file.close()

## Plots

### Covered area

In [ ]:
# Plotting
plot_width = 15 * cm
plot_height = 5 * cm
fig, ax = plt.subplots(
    1, 1, figsize=(plot_width, plot_height), constrained_layout=False, dpi=300
)
plt.grid(axis="both", color="0.8", linewidth=0.3)

# Plot points
shapely.plotting.plot_points(sp_slam_points, markersize=0.5, marker="o", color=(0.1, 0.1, 0.1), fillstyle="full")
shapely.plotting.plot_points(sp_odom_points, markersize=0.1, marker="o", color=(1.0, 0.1, 0.1), fillstyle="full")

# Plot sensor polygon
shapely.plotting.plot_polygon(
    sp_slam_hull.buffer(sensor_range * 0.25),
    add_points=False,
    color=(0, 0.5, 1),
    linewidth=0,
    alpha=0.1,
)
shapely.plotting.plot_polygon(
    sp_slam_hull.buffer(sensor_range * 0.50),
    add_points=False,
    color=(0, 0.5, 1),
    linewidth=0,
    alpha=0.1,
)
shapely.plotting.plot_polygon(
    sp_slam_hull.buffer(sensor_range * 0.75),
    add_points=False,
    color=(0, 0.5, 1),
    linewidth=0,
    alpha=0.1,
)
shapely.plotting.plot_polygon(
    sp_slam_hull.buffer(sensor_range * 1.00),
    add_points=False,
    color=(0, 0.5, 1),
    linewidth=0,
    alpha=0.1,
)
# shapely.plotting.plot_polygon(hull, add_points=False)

ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
ax.margins(x=0.1, y=0.1)

### Robot velocity

In [ ]:
smoothing_window="1000ms"
linewidth = 0.8

fig, ax = plt.subplots(
    1, 1, figsize=(plot_width, plot_height), constrained_layout=False, dpi=300
)

plt.plot(
    df_state_twist.index.total_seconds(),
    df_state_twist["lin_speed"].rolling(smoothing_window, center=True).mean(),
    label="Robot speed",
    linewidth=linewidth,
)
plt.plot(
    df_reference_twist.index.total_seconds(),
    df_reference_twist["lin_speed"].rolling(smoothing_window, center=True).mean(),
    label="Reference command",
    linewidth=linewidth,
)
plt.plot(
    df_operator_twist.index.total_seconds(),
    df_operator_twist["lin_speed"].rolling(smoothing_window, center=True).mean(),
    label="Operator command",
    linewidth=linewidth,
)

ax.legend(edgecolor=(1, 1, 1, 0), framealpha=0.9, loc=(0, 1.1), ncol=2)
# ax.set_title('Computation Time')
ax.set_xlabel("Time [s]")
ax.set_ylabel("Speed [m/s]")
ax.margins(x=0)

# Export
fig.set_tight_layout(True)
fig.savefig(os.path.join(mission_path, "mission_velocity.pdf"))

### Autonomy plot